In [ ]:
import pandas as pd
import time
import random
import google.generativeai as genai

API_KEY = ""
genai.configure(api_key=API_KEY)

INPUT_CSV = "C:\\Users\\HP\\Downloads\\colleges.csv"
OUTPUT_CSV = "colleges_with_states_generative6.csv"

def ask_gemini_state(college_name, max_retries=50):
    model = genai.GenerativeModel("gemini-2.0-flash")
    prompt = f"Which Indian state is '{college_name}' located in? Only return the state name."

    for attempt in range(max_retries):
        try:
            response = model.generate_content(prompt)
            return response.text.strip()
        except Exception as e:
            error_str = str(e)

            if "429" in error_str:
                # Look for retry delay from Gemini error response if available
                retry_seconds = 2 ** attempt + random.uniform(0, 1)
                if "retry_delay" in error_str:
                    try:
                        # Extract the retry delay from the string using simple parsing
                        # NOTE: For production, parse the structured error object if available
                        seconds_index = error_str.find("seconds:")
                        if seconds_index != -1:
                            retry_seconds = float(error_str[seconds_index:].split()[1])
                    except Exception:
                        pass  # fallback to exponential backoff

                print(f"⏳ Rate limit hit for '{college_name}'. Retrying in {retry_seconds:.2f} seconds...")
                time.sleep(retry_seconds)
            else:
                print(f"❌ Non-rate-limit error for {college_name}: {e}")
                break

    return "Error"

# Load the dataset
df = pd.read_csv(INPUT_CSV)
df.columns = [col.strip() for col in df.columns]

if "CollegeName" not in df.columns:
    raise ValueError("Input CSV must have a 'CollegeName' column")

states = []
for i, college in enumerate(df["CollegeName"], 1):
    print(f"[{i}/{len(df)}] Asking Gemini about: {college}")
    state = ask_gemini_state(college)
    print(f"    → Gemini says: {state}\n")
    states.append(state)

# Save the results
df["State"] = states
df.to_csv(OUTPUT_CSV, index=False)
print(f"✅ Done! Output saved to {OUTPUT_CSV}")

C:\Users\HP\AppData\Roaming\Python\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[1/538] Asking Gemini about: OTHERS
    → Gemini says: This question cannot be answered. "OTHERS" is not a location. You need to provide a valid location within India.

[2/538] Asking Gemini about: SMCE
    → Gemini says: Tamil Nadu

[3/538] Asking Gemini about: SREENIDHI UNIVERSITY
    → Gemini says: Telangana

[4/538] Asking Gemini about: KAKINADA INSTITUTE OF ENGINEERING AND TECHNOLOGY WOMEN'S
    → Gemini says: Andhra Pradesh

[5/538] Asking Gemini about: ALPHORES WOMEN'S DEGREE &PG COLLEGE KARIMNAGAR
    → Gemini says: Telangana

[6/538] Asking Gemini about: GITAM UNIVERSITY, BENGALURU
    → Gemini says: Karnataka

[7/538] Asking Gemini about: ALPHORES WOMENS
    → Gemini says: Telangana

[8/538] Asking Gemini about: MAHARAJA VIJAYARAM GAJAPATHIRAJ COLLEGE OF ENGINEERING
    → Gemini says: Andhra Pradesh

[9/538] Asking Gemini about: BITS PILANI, HYDERABAD CAMPUS
    → Gemini says: Telangana

[10/538] Asking Gemini about: ADITYA DEGREE COLLEGE FOR WOMEN
    → Gemini says: Andhra P

KeyboardInterrupt: 